In [ ]:
#VCF a FASTA
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

ins_vcf = "inserciones.vcf"
ins_fasta = "inserciones.fasta"

def vcf_to_fasta():
    records = []
    with open(ins_vcf) as vcf:
        for line in vcf:
            if line.startswith("#"): continue
            cols = line.strip().split("\t")
            if len(cols) < 5: continue

            chrom, pos, _, ref, alt = cols[:5]
            if "<INS>" in alt:
                continue
            if len(alt) > len(ref):
                insertion_seq = alt[len(ref):]
                seq_record = SeqRecord(
                    Seq(insertion_seq.strip()),
                    id=f"{chrom}:{pos}",
                    description=""
                )
                records.append(seq_record)

    with open(ins_fasta, "w") as output_handle:
        SeqIO.write(records, output_handle, "fasta")
    print(f"FASTA generado: {ins_fasta}")

if __name__ == "__main__":
    vcf_to_fasta()


In [ ]:
#Validación archivo fasta 
def validar_fasta(ruta_archivo):
    with open(ruta_archivo) as f:
        line_num = 0
        seq_started = False
        for line in f:
            line_num += 1
            line = line.strip()
            if line.startswith('>'):
                seq_started = True
            else:
                if not seq_started:
                    print(f"Error en línea {line_num}: Secuencia sin encabezado.")
                    return False
                if not line.isalpha():
                    print(f"Error en línea {line_num}: Secuencia contiene caracteres no alfabéticos.")
                    return False
    print("Archivo FASTA validado correctamente.")
    return True
# Llamas a la función pasando la ruta correcta
validar_fasta("/home/alumno16/TFM1/inserciones.fasta")

In [ ]:
#Filtrado 1
import pandas as pd

cols = [
    "qseqid", "sseqid", "pident", "length", "mismatch", "gapopen",
    "qstart", "qend", "sstart", "send", "evalue", "bitscore"
]

df = pd.read_csv(
    "resultados_inserciones_vs_exones.tsv",
    sep="\t",
    names=cols
)

# Filtro razonable: si pongo 200 pierdo el control positivo 
filtered = df[
    (df["pident"] >= 90) &
    (df["length"] >= 180) &
    (df["evalue"] <= 1e-5)
]

# Mejor hit por inserción (qseqid)
best_hits = (
    filtered
    .sort_values(["qseqid", "bitscore", "evalue"], ascending=[True, False, True])
    .drop_duplicates("qseqid", keep="first")
)

best_hits.to_csv("resultados_filtrados.tsv", sep="\t", index=False)


In [ ]:
#Cambio el tsv a csv para mejor manejo de datos
import pandas as pd
# Cambia el nombre por tu archivo
input_file = 'resultados_filtrados1.tsv'
output_file = 'resultados_procesados1.csv'
# Nombres de columnas que mencionaste
cols = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen',
        'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
# Leer archivo TSV
df = pd.read_csv(input_file, sep='\t', names=cols)
# 1) Separar qseqid en dos columnas nuevas: "chr" y "pos"
qseq_split = df['qseqid'].str.split(':', n=1, expand=True)
df['chr'] = qseq_split[0]
df['pos'] = qseq_split[1]
# 2) Separar sseqid en 4 columnas nuevas: ensg, enst, ense, resto
# Separar por '|'
sseq_split = df['sseqid'].str.split('|', expand=True)
df['ensg'] = sseq_split[0]
df['enst'] = sseq_split[1]
df['ense'] = sseq_split[2]
df['resto'] = sseq_split[3]
# Opcional: eliminar las columnas originales si quieres
df.drop(['qseqid', 'sseqid'], axis=1, inplace=True)
# Definir orden de columnas nuevas que quieres al inicio
cols_new = ['chr', 'pos', 'ensg', 'enst', 'ense', 'resto']
# Columnas restantes
cols_restantes = [col for col in df.columns if col not in cols_new]
# Reordenar DataFrame poniendo primero las nuevas
df = df[cols_new + cols_restantes]
# Ordenar por 'resto' ascendente
df = df.sort_values(by='resto')
# Guardar como CSV (puedes cambiar sep='\t' para TSV si prefieres)
df.to_csv(output_file, index=False)
print(f"Archivo procesado guardado en {output_file}")

In [ ]:
#Filtrado 2, inserciones que contengan 2 ó más exones
import pandas as pd

# Definir nombres de columnas según tu encabezado
cols = [
    "chr", "pos", "ensg", "enst", "ense", "genID",
    "pident", "length", "mismatch", "gapopen",
    "qstart", "qend", "sstart", "send",
    "evalue", "bitscore"
]

# Cargar CSV
df = pd.read_csv("resultados_procesados1.csv", names=cols, header=0)

# Agrupar por gen y posición y contar exones distintos
conteo_exones_por_pos = (
    df.groupby(["ensg", "pos"])["ense"]
    .nunique()
    .reset_index()
    .rename(columns={"ense": "n_exones_en_pos"})
)

# Filtrar posiciones con ≥ 2 exones
conteo_mas_de_1_exon = conteo_exones_por_pos[
    conteo_exones_por_pos["n_exones_en_pos"] >= 2
]

# Hacer merge con el DataFrame original para recuperar las demás columnas
# Nos quedamos solo con filas que tengan esos ensg y pos
df_filtrado = df.merge(
    conteo_mas_de_1_exon,
    on=["ensg", "pos"],
    how="inner"
)

# Opcional: quitar duplicados si solo quieres una fila por posición
df_filtrado = df_filtrado.drop_duplicates(
    subset=["chr", "pos", "ensg", "n_exones_en_pos"]
)

# Mostrar resultados
print(df_filtrado[["chr", "pos", "ensg", "n_exones_en_pos", "genID"]])

# Guardar si lo deseas
df_filtrado.to_csv("conteo_exones.csv", index=False)